# Plot Split Data

In [1]:
import functionsMasterProjectMeinhart as fmpm
from matplotlib.widgets import Button
from scipy.signal import butter, lfilter

## First load split data

In [2]:
subject_nums = ['01','02','03','04','05','07','08','09'] # subject 06, 10: no timetable available
exercise_abbrs  = ['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2']
repetitions = ['05','10','15']

file_dir = r'E:\Physio_Data_Split'

# create big nested dictionary for all data
allData_raw = {}

for sub_num in subject_nums:
    allData_raw[sub_num] = {}
    
    for ex_abb in exercise_abbrs:
        allData_raw[sub_num][ex_abb] = {}
        
        for rep in repetitions:
            allData_raw[sub_num][ex_abb][rep] = {}
            
            # put file name together
            file_name = 'subject' + sub_num + '_' + ex_abb + '_' + rep + '.csv'
            
            # join directory and file name
            file_path = os.path.join(file_dir, file_name)
            
            # use written function to get data from corresponding csv-file
            allData_raw[sub_num][ex_abb][rep] =  fmpm.get_sensor_data(in_file=file_path, 
                                                 signals=['Acc','Gyr','Mag'], 
                                                 sampling_rate=256, 
                                                 start_time=None, 
                                                 stop_time=None, 
                                                 skip_rows=0, 
                                                 sep=',',
                                                 add_info=file_name)

## Filter data

In [3]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data, axis=0)
    return y


# filter requirements
order = 6 # order of filter
sampling_rate = 256 # [Hz]
cutoff = 10 # cutoff frequency of the filter [Hz]


allData_filt = {}

for sub_num in subject_nums:
    allData_filt[sub_num] = {}
    
    for ex_abb in exercise_abbrs:
        allData_filt[sub_num][ex_abb] = {}
        
        for rep in repetitions:
            allData_filt[sub_num][ex_abb][rep] = {}
            
            allData_filt[sub_num][ex_abb][rep]['time'] = allData_raw[sub_num][ex_abb][rep]['time']
            
            for signal in ['Acc','Gyr','Mag']:
                allData_filt[sub_num][ex_abb][rep][signal] = butter_lowpass_filter(allData_raw[sub_num][ex_abb][rep][signal], 
                                                                                   cutoff, sampling_rate, order)


## Plot split data

In [4]:
%matplotlib auto

# Select data to plot:
#allData = allData_raw
allData = allData_filt


# dictionary for exercises
exercise = {}
exercise['RF'] = 'Raises Front'
exercise['RO'] = 'Raises Oblique'
exercise['RS'] = 'Raises Side'
exercise['LR'] = 'Rotation Wrist'
exercise['BC'] = 'Biceps Curls'
exercise['TC'] = 'Triceps Curls'
exercise['MP'] = 'Military Press'
exercise['SA'] = 'Shoulder Adduct.'
exercise['P1'] = 'PNF Diagonal 1'
exercise['P2'] = 'PNF Diagonal 2'


fig, axs = plt.subplots(3,2, sharex=True, figsize=(18,9))
plt.subplots_adjust(bottom=0.2, hspace=0.4) # make space for buttons


class Index(object):
    
    ind_sub = 0
    ind_ex = 0

    def next_subject(self, event):
        self.ind_sub = (self.ind_sub + 1) % len(subject_nums)
        self.plot_signals()

    def prev_subject(self, event):
        self.ind_sub = (self.ind_sub - 1) % len(subject_nums)
        self.plot_signals()
        
    def next_exercise(self, event):
        self.ind_ex = (self.ind_ex + 1) % len(exercise_abbrs)
        self.plot_signals()

    def prev_exercise(self, event):
        self.ind_ex = (self.ind_ex - 1) % len(exercise_abbrs)
        self.plot_signals()
        
    def plot_signals(self):
                
        fig.suptitle('Subject: ' + subject_nums[self.ind_sub] + \
                     '     Exercise: ' + exercise[exercise_abbrs[self.ind_ex]], fontsize=22, fontweight='bold')
       
        # plot linear acceleration
        plt.sca(axs[0,0])
        axs[0,0].clear()
        fmpm.plot_signal(allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[0]]['Acc'],
                         allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[0]]['time'],
                         Title='Acceleration ('+repetitions[0]+' rep.)',
                         xLabel='')
        
        plt.sca(axs[1,0])
        axs[1,0].clear()
        fmpm.plot_signal(allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[1]]['Acc'],
                         allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[1]]['time'],
                         Title='Acceleration ('+repetitions[1]+' rep.)',
                         xLabel='')
        
        plt.sca(axs[2,0])
        axs[2,0].clear()
        fmpm.plot_signal(allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[2]]['Acc'],
                         allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[2]]['time'],
                         Title='Acceleration ('+repetitions[2]+' rep.)')

        # plot angular velocity
        plt.sca(axs[0,1])
        axs[0,1].clear()
        fmpm.plot_signal(allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[0]]['Gyr'],
                         allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[0]]['time'],
                         Title='Angular Velocity ('+repetitions[0]+' rep.)',
                         yLabel=r'$vel \enspace [\frac{deg}{s}]$',
                         xLabel='')
        
        plt.sca(axs[1,1])
        axs[1,1].clear()
        fmpm.plot_signal(allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[1]]['Gyr'],
                         allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[1]]['time'],
                         Title='Angular Velocity ('+repetitions[1]+' rep.)',
                         yLabel=r'$vel \enspace [\frac{deg}{s}]$',
                         xLabel='')
        
        plt.sca(axs[2,1])
        axs[2,1].clear()
        fmpm.plot_signal(allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[2]]['Gyr'],
                         allData[subject_nums[self.ind_sub]][exercise_abbrs[self.ind_ex]][repetitions[2]]['time'],
                         Title='Angular Velocity ('+repetitions[2]+' rep.)',
                         yLabel=r'$vel \enspace [\frac{deg}{s}]$')
        
        plt.draw()


callback = Index()
callback.plot_signals()

ax_prev_sub = plt.axes([0.57, 0.05, 0.15, 0.075])
ax_next_sub = plt.axes([0.74, 0.05, 0.15, 0.075])
ax_prev_ex =  plt.axes([0.15, 0.05, 0.15, 0.075])
ax_next_ex =  plt.axes([0.32, 0.05, 0.15, 0.075])

b_next_sub = Button(ax_next_sub, 'Next Subject')
b_next_sub.on_clicked(callback.next_subject)
b_prev_sub = Button(ax_prev_sub, 'Previous Subject')
b_prev_sub.on_clicked(callback.prev_subject)

b_next_ex = Button(ax_next_ex, 'Next Exercise')
b_next_ex.on_clicked(callback.next_exercise)
b_prev_ex = Button(ax_prev_ex, 'Previous Exercise')
b_prev_ex.on_clicked(callback.prev_exercise)

plt.show()

Using matplotlib backend: TkAgg
